In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
import statsmodels.api as sm
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt


In [ ]:
# Fetch data
market_data = pd.read_csv('sap500.csv')

market_data['Date'] = pd.to_datetime(market_data['Date'])

market_data = market_data.sort_values(by='Date')  # Sort by Date if not already sorted

In [ ]:
market_data.head()

In [ ]:
# Plot the time series
plt.figure(figsize=(12, 6))
plt.plot(market_data['Date'], market_data['Close'], label="Close Price", color='b')

# Formatting the plot
plt.xlabel("Date")
plt.ylabel("Close Price")
plt.title("Market Data Close Price Time Series")
plt.legend()
plt.grid(True)

# Display the plot
plt.show()

In [ ]:
market_data = market_data[market_data['Date'] >= '1990-01-01']

# Plot the time series
plt.figure(figsize=(12, 6))
plt.plot(market_data['Date'], market_data['Close'], label="Close Price", color='b')

# Formatting the plot
plt.xlabel("Date")
plt.ylabel("Close Price")
plt.title("Market Data Close Price Time Series")
plt.legend()
plt.grid(True)

# Display the plot
plt.show()

In [ ]:
market_data.set_index('Date', inplace=True)

# Ensure period is set (252 trading days in a year)
stl = STL(market_data['Close'], period=252, robust=True)  
result = stl.fit()

# Plot the decomposition
fig, axs = plt.subplots(4, 1, figsize=(12, 8), sharex=True)

axs[0].plot(market_data.index, market_data['Close'], color='black', label="Original")
axs[0].set_title("Original Time Series")

axs[1].plot(market_data.index, result.trend, color='blue', label="Trend")
axs[1].set_title("Trend Component")

axs[2].plot(market_data.index, result.seasonal, color='green', label="Seasonal")
axs[2].set_title("Seasonal Component")

axs[3].plot(market_data.index, result.resid, color='red', label="Residual")
axs[3].set_title("Residual Component")
axs[3].axhline(0, linestyle='--', color='gray')

# Formatting
plt.tight_layout()
plt.show()

In [ ]:
# ADF test function
def adf_test(series, title=""):
    result = adfuller(series.dropna())  # Drop NaN values for differenced series
    print(f"ADF Test for {title}")
    print(f"ADF Statistic: {result[0]}")
    print(f"p-value: {result[1]}")
    print("Critical Values:", result[4])
    if result[1] <= 0.05:
        print("The series is stationary (reject H0).")
    else:
        print("The series is non-stationary (fail to reject H0).")
    print("-" * 50)

# Perform ADF test on original Close price
adf_test(market_data['Close'], title="Original Close Price")

# Apply first-order differencing
market_data['Close_diff'] = market_data['Close'].diff()

# Perform ADF test on differenced data
adf_test(market_data['Close_diff'], title="Differenced Close Price")

# Plot the original and differenced series
fig, axs = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

axs[0].plot(market_data.index, market_data['Close'], color='black', label="Original Close Price")
axs[0].set_title("Original Close Price Time Series")
axs[0].legend()

axs[1].plot(market_data.index, market_data['Close_diff'], color='blue', label="First-Order Differenced")
axs[1].set_title("Differenced Close Price Time Series")
axs[1].legend()

plt.tight_layout()
plt.show()


In [ ]:
# First-order differencing to make the series stationary
market_data['Close_diff'] = market_data['Close'].diff().dropna()

# Calculate ACF and PACF values
acf_values = acf(market_data['Close_diff'].dropna(), nlags=50)
pacf_values = pacf(market_data['Close_diff'].dropna(), nlags=50)

# Print ACF and PACF values
print("ACF Values:\n", acf_values)
print("\nPACF Values:\n", pacf_values)

# Plot ACF and PACF for the differenced data
fig, axs = plt.subplots(2, 1, figsize=(12, 8))

plot_acf(market_data['Close_diff'].dropna(), lags=50, ax=axs[0])  # Autocorrelation function
axs[0].set_title("Autocorrelation Function (ACF)")

plot_pacf(market_data['Close_diff'].dropna(), lags=50, ax=axs[1])  # Partial autocorrelation function
axs[1].set_title("Partial Autocorrelation Function (PACF)")

plt.tight_layout()
plt.show()

# Run Ljung-Box test on the differenced data (checking for autocorrelation)
# ljung_box_results = acorr_ljungbox(market_data['Close_diff'].dropna(), lags=[10, 20, 30], return_df=True)
# print(ljung_box_results)


In [ ]:
# Sort the DataFrame by date (in case it is not sorted)
market_data.sort_index(inplace=True)

# Reindex the DataFrame with a complete date range
# This will insert missing dates as NaN for close prices
date_range = pd.date_range(start=market_data.index.min(), end=market_data.index.max(), freq='B')  # 'B' for business days
market_data_reindexed = market_data.reindex(date_range)

# Interpolate missing data points (if necessary)
# Linear interpolation is a common approach for filling missing data
market_data_reindexed['Close'] = market_data_reindexed['Close'].interpolate(method='linear')

# Split the data: train = before 2024, test = from 2024 onward
train = market_data_reindexed[market_data_reindexed.index < '2024-01-01']
test = market_data_reindexed[market_data_reindexed.index >= '2024-01-01']

In [ ]:
# # Function to find the best AR order
# def find_best_ar_order(train, max_lags=10):
#     best_aic = np.inf
#     best_order = None
#     best_model = None
    
#     # Loop through different lags
#     for lag in [10, 365]:
#         model = ARIMA(train['Close'], order=(lag, 0, 0))  # AR model
#         model_fitted = model.fit()
#         aic = model_fitted.aic
        
#         # Update the best model based on AIC
#         if aic < best_aic:
#             best_aic = aic
#             best_order = lag
#             best_model = model_fitted
    
#     return best_order, best_model

# # Find the best AR order using AIC
# best_order, best_model = find_best_ar_order(train)

# # Print the best AR order
# print(f"Best AR Order: {best_order}")

# Make predictions on the test set
lag = 365
model = AutoReg(train['Close'], lags=lag)  # Using 1 lag (AR(1) model)
model_fitted = model.fit()

# Predict on the test set
predictions = model_fitted.predict(start=test.index[0], end=test.index[-1], dynamic=False)


# Plot the actual vs predicted values
plt.figure(figsize=(10, 6))
plt.plot(test.index, test['Close'], label='Actual', color='blue')
plt.plot(test.index, predictions, label='Predicted', color='red')
plt.xlabel('Time')
plt.ylabel('Price')
plt.title(f'Actual vs Predicted Prices (AR({lag}) Model)')
plt.legend()
plt.show()

# Calculate MSE, MAE, and MAPE
mse = mean_squared_error(test['Close'], predictions)
mae = mean_absolute_error(test['Close'], predictions)
mape = np.mean(np.abs((test['Close'] - predictions) / test['Close'])) * 100

# Show evaluation metrics
print(f'Mean Squared Error (MSE): {mse}')
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Absolute Percentage Error (MAPE): {mape}%')

In [ ]:
# Train the ARIMA model with MA(1)
lag = 365
model_ma = ARIMA(train['Close'], order=(0, 0, lag))  # MA(1) model
model_fitted_ma = model_ma.fit()

# Make predictions on the test set
predictions = model_fitted_ma.forecast(steps=len(test))

# Plot the actual vs predicted values
plt.figure(figsize=(10, 6))
plt.plot(test.index, test['Close'], label='Actual', color='blue')
plt.plot(test.index, predictions, label='Predicted', color='red')
plt.xlabel('Time')
plt.ylabel('Price')
plt.title('Actual vs Predicted Prices (MA Model)')
plt.legend()
plt.show()

# Calculate MSE, MAE, and MAPE
mse = mean_squared_error(test['Close'], predictions)
mae = mean_absolute_error(test['Close'], predictions)
mape = np.mean(np.abs((test['Close'] - predictions) / test['Close'])) * 100

# Show evaluation metrics
print(f'Mean Squared Error (MSE): {mse}')
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Absolute Percentage Error (MAPE): {mape}%')

# Show model details in plain English
print("\nModel Details (MA Model):")
print("We trained a Moving Average (MA) model, specifically an ARIMA model with the order (0, 0, 1). This means the model uses only the moving average of the past 1 observation to predict the next value. The MA(1) model assumes that the current observation is linearly dependent on the error from the previous time step.")

In [ ]:
# Train the ARIMA model with automatic selection of p, d, q
# For daily data, assume d=1 for differencing and find the best p and q
# Let's use ACF and PACF plots to choose p and q

# ARIMA(p, d, q) - In this example, d=1 (since most daily financial data needs 1st differencing to make stationary)
# For p and q, let's use a range and compare the models.

# Set the ARIMA model parameters
p, d, q = 1, 1, 1  # Initial guess for AR, I, MA parameters

# Train the ARIMA model
model_arima = ARIMA(train['Close'], order=(p, d, q))
model_fitted_arima = model_arima.fit()

# Make predictions on the test set
predictions = model_fitted_arima.forecast(steps=len(test))

# Plot the actual vs predicted values
plt.figure(figsize=(10, 6))
plt.plot(test.index, test['Close'], label='Actual', color='blue')
plt.plot(test.index, predictions, label='Predicted', color='red')
plt.xlabel('Time')
plt.ylabel('Price')
plt.title('Actual vs Predicted Prices (ARIMA Model)')
plt.legend()
plt.show()

# Calculate MSE, MAE, and MAPE
mse = mean_squared_error(test['Close'], predictions)
mae = mean_absolute_error(test['Close'], predictions)
mape = np.mean(np.abs((test['Close'] - predictions) / test['Close'])) * 100

# Show evaluation metrics
print(f'Mean Squared Error (MSE): {mse}')
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Absolute Percentage Error (MAPE): {mape}%')

# Show model details in plain English
print("\nModel Details (ARIMA Model):")
print("We trained an ARIMA model, which is composed of three parts:")
print(f"- **AR(p)**: Autoregressive term, where 'p' is the number of lags of the dependent variable used to predict the current value.")
print(f"- **I(d)**: Integrated term, where 'd' is the degree of differencing used to make the time series stationary. For daily close price data, we used d=1 to remove trend.")
print(f"- **MA(q)**: Moving average term, where 'q' is the number of lags of forecast errors used to predict the current value.")
print(f"In this model, we used AR(1), I(1), and MA(1) to capture the autoregressive, integrated, and moving average effects.")

In [ ]:
# Function to find the best SARIMA order
def find_best_sarima_order(train, max_P=2, max_D=2, max_Q=2, seasonal_period=12):
    best_aic = np.inf
    best_order = None
    best_model = None
    
    # Loop through different combinations of (P, D, Q) and try seasonal differencing
    for P in range(0, max_P + 1):
        for D in range(0, max_D + 1):
            for Q in range(0, max_Q + 1):
                try:
                    model = SARIMAX(train['Close'], order=(P, D, Q), seasonal_order=(P, D, Q, seasonal_period))
                    model_fitted = model.fit(disp=False)
                    aic = model_fitted.aic
                    
                    # Update the best model based on AIC
                    if aic < best_aic:
                        best_aic = aic
                        best_order = (P, D, Q, seasonal_period)
                        best_model = model_fitted
                except Exception as e:
                    print(f"Error with SARIMA order ({P}, {D}, {Q}, {seasonal_period}): {e}")
                    continue
    
    return best_order, best_model

# Find the best SARIMA order using AIC
best_order, best_model = find_best_sarima_order(train)

# Print the best SARIMA order
print(f"Best SARIMA Order: {best_order}")

# Make predictions on the test set
predictions = best_model.forecast(steps=len(test))

# Plot the actual vs predicted values
plt.figure(figsize=(10, 6))
plt.plot(test.index, test['Close'], label='Actual', color='blue')
plt.plot(test.index, predictions, label='Predicted', color='red')
plt.xlabel('Time')
plt.ylabel('Price')
plt.title(f'Actual vs Predicted Prices (SARIMA{best_order})')
plt.legend()
plt.show()

# Calculate MSE, MAE, and MAPE
mse = mean_squared_error(test['Close'], predictions)
mae = mean_absolute_error(test['Close'], predictions)
mape = np.mean(np.abs((test['Close'] - predictions) / test['Close'])) * 100

# Show evaluation metrics
print(f'Mean Squared Error (MSE): {mse}')
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Absolute Percentage Error (MAPE): {mape}%')

# Show model details in plain English
print("\nModel Details (SARIMA Model):")
print(f"The best SARIMA order identified by the model is {best_order}. This means that the model uses:")
print(f"- P={best_order[0]} seasonal autoregressive terms (AR),")
print(f"- D={best_order[1]} seasonal differencing terms to make the series stationary,")
print(f"- Q={best_order[2]} seasonal moving average terms (MA),")
print(f"- s={best_order[3]} seasonal cycle length (e.g., 12 months for yearly seasonality).")
print("SARIMA is an extension of ARIMA that supports seasonal effects, which are common in time series data with periodic patterns.")


In [ ]:
# Function to train the ETS model and make predictions
def train_ets_model(train, seasonal_period=365):  # Updated for daily data
    model = ExponentialSmoothing(train['Close'], 
                                 trend='add', 
                                 seasonal='add', 
                                 seasonal_periods=seasonal_period)
    model_fitted = model.fit()
    return model_fitted

# Train the ETS model
ets_model = train_ets_model(train)

# Make predictions on the test set
predictions = ets_model.forecast(len(test))

# Plot the actual vs predicted values
plt.figure(figsize=(10, 6))
plt.plot(test.index, test['Close'], label='Actual', color='blue')
plt.plot(test.index, predictions, label='Predicted', color='red')
plt.xlabel('Time')
plt.ylabel('Price')
plt.title(f'Actual vs Predicted Prices (ETS)')
plt.legend()
plt.show()

# Calculate MSE, MAE, and MAPE
mse = mean_squared_error(test['Close'], predictions)
mae = mean_absolute_error(test['Close'], predictions)
mape = np.mean(np.abs((test['Close'] - predictions) / test['Close'])) * 100

# Show evaluatio
